# Python examples

In this notebook, I show several different ways to estimate a linear regression in Python

## Linear regression

### Pure python approach

see formula for OLS [here](https://en.wikipedia.org/wiki/Simple_linear_regression)

In [12]:
# x and y data as python lists

x = [3,4,5,6,7,8]
y = [0,7,17,26,35,45]

n=len(x) # length of x

In [12]:
# an example of `assert` statement, see what happens when you add another value in either x or y above
assert len(x) == len(y), f'x and y are not of equal length (len(x)={len(x)}, len(y)={len(y)})'

In [12]:
# initialize all accumulators we need (check the expressions for a and b in the link above )
sumx = sumxy = sumx2 = sumy = 0
for i in range(n):
    sumx += x[i]
    sumx2 += x[i]**2
    sumy += y[i]
    sumxy += x[i]*y[i]
    

In [12]:
# compute averages    
xm = sumx/n
ym = sumy/n

In [12]:
# compute bhat and ahat
bhat = (sumxy - xm*sumy - ym*sumx + n*xm*ym)/(sumx2 - n*xm**2)
ahat = ym - (bhat*xm)

In [13]:
# an example of a print statement using a multiline f-string
print(f'''
the estimate of ahat is {ahat},
the estimate of bhat is {bhat}
          ''')
# add formatting to round the numbers
print(f'''
the estimate of ahat is {ahat:.2f},
the estimate of bhat is {bhat:.2f}
          ''')


the estimate of ahat is -28.3047619047619,
the estimate of bhat is 9.085714285714285
          

the estimate of ahat is -28.30,
the estimate of bhat is 9.09
          


### Using numpy to vectorize the loop

In [2]:
# import numpy 
import numpy as np

In [3]:
# transform x and y from lists to numpy arrays
xnp = np.array(x)
ynp = np.array(y)

In [4]:
# version 1 - replicating the steps in the pure python version

sumx = np.sum(xnp) 
sumxy = np.sum(xnp*ynp)
sumx2 = np.sum(xnp**2)
sumy = np.sum(ynp)
    
xm = sumx/n
ym = sumy/n

# compute bhat and ahat
bhat_np = (sumxy - xm*sumy)/(sumx2 - n*xm**2)
ahat_np = ym - (bhat_np*xm)

assert ahat_np == ahat
assert bhat_np == bhat

In [5]:
# version 2 - replace the summation in the OLS formula for beta with dot products 
xm = np.mean(xnp)
ym = np.mean(ynp)

bhat_np2 = (xnp-xm)@(ynp-ym)/((xnp-xm)@(xnp-xm)) 
ahat_np2 = ym - bhat_np2*xm # same as above

assert ahat_np2 == ahat_np
assert bhat_np2 == bhat_np

### Using numpy and matrix algebra approach

* (see matrix formulation for OLS [here](https://en.wikipedia.org/wiki/Ordinary_least_squares#Matrix/vector_formulation), and explanation of **design matrix** [here](https://en.wikipedia.org/wiki/Design_matrix))

In [6]:
# create design matrix and reshape y
X_mat = np.hstack((np.ones((n,1)), xnp.reshape(-1,1)))
Y_mat = ynp.reshape((-1,1)) # make y into a column vector


print(f'Y_mat is:\n {Y_mat}')
print(f'\n\nX_mat is:\n {X_mat}')

Y_mat is:
 [[ 0]
 [ 7]
 [17]
 [26]
 [35]
 [45]]


X_mat is:
 [[1. 3.]
 [1. 4.]
 [1. 5.]
 [1. 6.]
 [1. 7.]
 [1. 8.]]


In [7]:
# compute Bhat = [ahat, bhat]
Bhat = np.linalg.inv(X_mat.T@X_mat) @ (X_mat.T@ Y_mat)
print(f'Bhat is:\n {Bhat}')

Bhat is:
 [[-28.3047619 ]
 [  9.08571429]]


* check the values are (almost) equal to the ones found above (see [numpy doc](https://numpy.org/doc/stable/reference/routines.testing.html) for more details on how to compare numpy arrays)

In [8]:
np.testing.assert_allclose(Bhat, 
                           np.array([ahat_np, bhat_np]).reshape(-1,1))

Bhat above solves the following matrix equation: 

$$ X'Y = (X'X) \mathbf{\beta} => \mathbf{\beta} = (X'X)^{-1} (X'Y) $$

numpy has a function for solving such equations

In [9]:
XX = X_mat.T @ X_mat
XY = X_mat.T @ Y_mat

Bhat2 = np.linalg.solve(XX, XY)
Bhat2

array([[-28.3047619 ],
       [  9.08571429]])

* numpy also has a dedicated function for solving least squares problems (see [numpy doc](https://numpy.org/doc/stable/reference/generated/numpy.linalg.lstsq.html?highlight=least%20squares) for details)
* in addition to the solution, the function returns the sum of squared residuals, the rank and singular values of the design matrix. If we are not interested in those outputs, it is conventional to use `_` for each one
* alternatively, we can get the full output and then keep the first element, which is the parameter estimates

In [10]:
Bhat3, _, _, _ = np.linalg.lstsq(X_mat, Y_mat, rcond=None)

In [11]:
output =  np.linalg.lstsq(X_mat, Y_mat, rcond=None)
Bhat3 = output[0]
Bhat3

array([[-28.3047619 ],
       [  9.08571429]])

### Conclusion

* There a many other ways to get OLS in Python
    * `linear_regression` in the statistics module of the Python standard library (see [statistics doc](https://docs.python.org/3/library/statistics.html#statistics.linear_regression))
    * `scipy.stats.linregress` from the scipy library (see [scipy doc](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html))
    * `OLS` from the statsmodels library (see [statsmodels doc](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html))
    * and so on (with other libraries which we are not going to use in this course)